In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# import setuptools.dist
import undetected_chromedriver as uc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(1)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists SaldytuvaiPigu
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists SaldytuvaiPigu
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [3]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/saldytuvai?page=1'
# url = 'https://pigu.lt/lt/saldytuvai/'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [4]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'c-product-card__title'})
for linkas in linkai:
    link = linkas.find('a')['href']
    # print(link)
    p_nuorodos.append(link)

print(len(p_nuorodos))
print(p_nuorodos)


max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
print(max_page_number)



59
['https://pigu.lt/lt/saldytuvai/gorenje-rf212epw4?id=80913785', 'https://pigu.lt/lt/saldytuvai/beko-rcsa270k40sn?id=76458632', 'https://pigu.lt/lt/saldytuvai/beko-b3rcna364hxb?id=60394178', 'https://pigu.lt/lt/saldytuvai/beko-aeroflow-b1rcna404g?id=71955821', 'https://pigu.lt/lt/saldytuvai/beko-rcsa300k40wn?id=76458672', 'https://pigu.lt/lt/saldytuvai/schlosser-rfd230bwe?id=90495841', 'https://pigu.lt/lt/saldytuvai/midea-mdrd142fge01?id=89986255', 'https://pigu.lt/lt/saldytuvai/gorenje-r44e4w4?id=92695157', 'https://pigu.lt/lt/saldytuvai/beko-ts190340n?id=76459917', 'https://pigu.lt/lt/saldytuvai/beko-rcsa240k40wn?id=76458557', 'https://pigu.lt/lt/saldytuvai/schlosser-rfd162bs?id=24548705', 'https://pigu.lt/lt/saldytuvai/beko-tse1284n?id=32178427', 'https://pigu.lt/lt/saldytuvai/beko-rcsa240k40sn?id=76458537', 'https://pigu.lt/lt/saldytuvai/hisense-rb224d4bwe?id=80905845', 'https://pigu.lt/lt/saldytuvai/hisense-rb291d4cde?id=85648160', 'https://pigu.lt/lt/saldytuvai/whirlpool-w5721e

In [7]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://pigu.lt/lt/saldytuvai/gorenje-rf212epw4?id=80913785


In [8]:
bs = BeautifulSoup(source, 'html.parser')
firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
print(firstInfo)

secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
print(secondInfo)

# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

kaina = bs.find('div', {'class':'c-product__media'})
price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
print(kaina2)

# if 'price' in price['dataLayerItem']:
#     kaina2 = 'nera'
# else:
#     kaina2 = price['dataLayerItem']['price']
# print(kaina2)


# kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
# print(kaina)

specifikacija_keys = []
specifikacija_values = []
specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
for trow in specifikacija:
    td = trow.find_all('td')
    key = td[0].text.strip()
    value = td[1].text.strip()
    specifikacija_keys.append(key)
    specifikacija_values.append(value)

print(len(specifikacija_keys), specifikacija_keys)
print(len(specifikacija_values), specifikacija_values)




Laisvai pastatomas, Šaldytuvas su šaldikliu viršuje Gorenje RF212EPW4, Rankinis atitirpinimas
Gorenje RF212EPW4
198.49
34 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Gamintojo kodas:', 'Šaldytuvo tipas:', 'Rūšis:', 'Energijos klasė:', 'Energijos sąnaudos per metus:', 'Bendra talpa:', 'Šaldytuvo talpa:', 'Šaldiklio talpa:', 'Temperatūra šaldiklyje:', 'Šaldytuvo atitirpinimas:', 'Šaldiklio atitirpinimas:', 'Šaldiklio efektyvumas be elektros srovės:', 'Užšaldymo galia:', 'Klimato klasė:', 'Maksimalus triukšmo lygis:', 'Montavimo tipas:', 'Atostogų režimas:', 'Valdymas:', 'Papildomos funkcijos:', 'Plotis:', 'Gylis:', 'Svoris:', 'Lentynėlė buteliams:', 'Vidaus apšvietimas:', 'Atvertų durų garsinis signalas:', 'Pardavėjas:', 'Aukštis:', 'Prekės ženklas:', 'Spalva:', 'Visos šio ženklo prekės:']
34 ['80913785', 'Šaldytuvai', '1 vnt.', '0,52 x 0,55 x 1,26 m, 29,5 kg', 'RF212EPW4', 'Šaldytuvas su šaldikliu viršuje', 'Gorenje, Šaldytuvas su šal

In [ ]:
# createMainTable('VarlePigu.db')

True

In [11]:
# Saldytuvai Pigu scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
# driver = webdriver.Chrome(options=opcijos)
url = 'https://pigu.lt/lt/saldytuvai?page=16'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
last_page = int(max_page_number)

counts = 856
psl = 16
# iki_kokio_psl = 2
# while psl < iki_kokio_psl:
while psl < last_page + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []
    linkai = bs.find_all('div', {'class':'c-product-card__title'})
    for linkas in linkai:
        link = linkas.find('a')['href']
        # print(link)
        p_nuorodos.append(link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        # time.sleep(random.randint(5,10))
        time.sleep(3)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='SaldytuvaiPigu', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs = BeautifulSoup(source, 'html.parser')
        firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
        duomenys['firstInfo'] = firstInfo

        secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
        duomenys['secondInfo'] = secondInfo

        # kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
        # print(kaina)

        kaina = bs.find('div', {'class':'c-product__media'})
        price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
        kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
        duomenys['kaina'] = kaina2

        specifikacija_keys = []
        specifikacija_values = []
        specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
        for trow in specifikacija:
            td = trow.find_all('td')
            key = td[0].text.strip()
            value = td[1].text.strip()
            specifikacija_keys.append(key)
            specifikacija_values.append(value)

        for k, v in zip(specifikacija_keys, specifikacija_values):
            duomenys[k] = v
            
        if kaina2 == 'nenurodyta':
            continue
        else:
            cols_i_need = list(duomenys.keys())
        
            data = tuple(duomenys.values())
            
            cols_i_have = getColNamesList(table='SaldytuvaiPigu', dbname='VarlePigu.db')
            
            alterTableAddColumn(cols_i_need, cols_i_have, 'SaldytuvaiPigu', 'VarlePigu.db')
            
            writeMany(data, 'SaldytuvaiPigu', 'VarlePigu.db')
            print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
            counts = counts +1
            
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= last_page:
        next_nuoroda = f'https://pigu.lt/lt/saldytuvai?page={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
        
driver.quit()
print('Baigta!!!')


16 https://pigu.lt/lt/saldytuvai/liebherr-ire-5100-pure?id=39254648
Įrašytas įrašas Nr 856, puslapis 16
16 https://pigu.lt/lt/saldytuvai/liebherr-icse-5122?id=39296118
Įrašytas įrašas Nr 857, puslapis 16
16 https://pigu.lt/lt/saldytuvai/samsung-brb26605fwwef?id=40271203
Įrašytas įrašas Nr 858, puslapis 16
16 https://pigu.lt/lt/saldytuvai/samsung-brb30715ewwef?id=40271208
Įrašytas įrašas Nr 859, puslapis 16
16 https://pigu.lt/lt/saldytuvai/electrolux-lrb3de18s?id=40853278
Įrašytas įrašas Nr 860, puslapis 16
16 https://pigu.lt/lt/saldytuvai/bosch-kir81ade0?id=41395053
Įrašytas įrašas Nr 861, puslapis 16
16 https://pigu.lt/lt/saldytuvai/guzzanti-gz-44gw?id=42275813
Įrašytas įrašas Nr 862, puslapis 16
16 https://pigu.lt/lt/saldytuvai/sharp-sjba05dtxwfeu?id=43299233
Įrašytas įrašas Nr 863, puslapis 16
16 https://pigu.lt/lt/saldytuvai/bosch-ksv29nwep?id=43312753
Įrašytas įrašas Nr 864, puslapis 16
16 https://pigu.lt/lt/saldytuvai/samsung-rr39m7130s9eo?id=43335748
Įrašytas įrašas Nr 865, pusl